In [1]:
import os, email, re
import imaplib
import datetime
import psycopg2, yaml
import pandas as pd
import numpy as np
from imapclient import IMAPClient
from timeit import default_timer as timer

# Instantiate connection with MailServer & Postgres
with open ("secret.yml", 'r') as f:
    data = yaml.full_load(f)

param_dic = {
    "host"      : data.get('host'),
    "database"  : data.get('database'),
    "user"      : data.get('user'),
    "password"  : data.get('password')
}

EMAIL_UN = os.environ['UNAME']
EMAIL_PW = os.environ['UPASS']
server = IMAPClient('imap.gmail.com', use_uid=True, ssl=True)


# Establish Connection
server.login(EMAIL_UN, EMAIL_PW)

# Create DataFrame to load fetched emails
email_df = pd.DataFrame(columns=['MSG_ID','From','Subject','DATE_RECIEVED'])

# Select Folder
select_info = server.select_folder('[Gmail]/Spam')
#print('%d messages in INBOX' % select_info[b'EXISTS'])

# Filter Search
messages = server.search(['ALL'])
#print("%d messages" % len(messages))

# Fetched All Data based on the searched criteria
for msgid, data in server.fetch(messages, ['ENVELOPE','RFC822']).items():
    try:
        envelope = data[b'ENVELOPE']
        email_message = email.message_from_bytes(data[b'RFC822'])
        new_row = pd.Series({"MSG_ID":msgid, "From":email_message.get("From") ,"Subject":envelope.subject.decode(), "DATE_RECIEVED":envelope.date})
        email_df = email_df._append(new_row, ignore_index=True)
        print('Email Fetched: ID-NO:%d: "%s" Received@: %s' % (msgid, envelope.subject.decode(), envelope.date))
    except:
        new_row = pd.Series({"MSG_ID":msgid, "From":email_message.get("From"), "Subject":'Not Able to Decode', "DATE_RECIEVED":envelope.date})
        email_df = email_df._append(new_row, ignore_index=True)
        print('Email Failed: Can`t Decode')

# Perform Reworks on DataFrame to prepare for loading in PostgreSQL
def find_email(text):
    email = re.findall(r'[\w\.-]+@[\w\.-]+',str(text))
    return ",".join(email)
email_df['From']=email_df['From'].apply(lambda x: find_email(x))
#print("\Extracting email from dataframe columns:")
#print(email_df['sender_email'])

df = email_df
df = df.rename(columns={
    "MSG_ID": "msg_id",
    "From": "sender",
    "Subject": "subject",
    "DATE_RECIEVED": "date_receive"
})
df = df.replace(np.nan, 'N/A', regex=True)

# Connect and Load the DATA in PostgreSQL

# This function will allow us to connect to the database
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
    
conn = connect(param_dic)
# Function to execute any query in the database
def execute_query(conn, query):
    """ Execute a single query """
    
    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret
# Here we use the execute_many operation
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

# Run the execute_many strategy
execute_many(conn, df, 'spam_emails')

Email Fetched: ID-NO:943: "[SUBMISSION PENDING] FORM 206 - James Elliot Ciano (188596235)" Received@: 2023-12-28 14:33:28
Email Fetched: ID-NO:944: "James Elliot, Submission required for form ADF-037" Received@: 2023-12-29 14:41:34
Email Fetched: ID-NO:945: "FORM 206 - James Elliot Ciano (188596235) - SUBMISSION PENDING" Received@: 2023-12-30 14:32:18
Email Fetched: ID-NO:946: "Zywave Thanks You for Applying" Received@: 2024-01-03 11:09:00
Email Fetched: ID-NO:947: "=?UTF-8?Q?Harness_the_power_of_AI_=E2=9A=A1=EF=B8=8F_and?= =?UTF-8?Q?_boost_your_business_in_2024_=F0=9F=9A=80?=" Received@: 2024-01-03 11:33:04
Email Fetched: ID-NO:948: "RE: FORM 206 - James Elliot Ciano (188596235) - ACTION REQUIRED" Received@: 2024-01-04 14:04:22
Email Fetched: ID-NO:949: "=?UTF-8?Q?=F0=9F=8E=81_ASUS_Members_Exclusive_Gift_=F0=9F=8E=81_?= =?UTF-8?Q?Hi_Cianojameselliot,_automatically?= =?UTF-8?Q?_backup_your_computer_data_without?= =?UTF-8?Q?_fearing_your_files_getting_hacked.?=" Received@: 2024-01-05 03

/tmp/ipykernel_1345/2009084214.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  email_df = email_df._append(new_row, ignore_index=True)
